# 뉴스 및 커뮤니티 데이터 수집기

이 노트북은 다음 기능들을 제공합니다:

1. 구글 뉴스에서 원하는 키워드로 뉴스 기사 수집
2. 네이버 뉴스에서 뉴스 기사 수집
3. 디시인사이드, 네이버 카페 등 커뮤니티 게시글 수집
4. 수집된 텍스트에서 주요 개체(인물, 조직 등) 추출
5. 결과를 CSV 파일로 저장

## 사용 방법
1. 키워드를 설정하여 원하는 정보 검색
2. 수집 기간 설정 가능
3. 원하는 커뮤니티 선택 가능
4. 결과물은 CSV 파일로 저장됨


In [1]:
# 필요한 라이브러리 설치
%pip install GoogleNews newspaper3k transformers fugashi[unidic] pandas requests beautifulsoup4 selenium webdriver-manager



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/7.4 MB ? eta -:--:--
     ---------------------------- ----------- 5.2/7.4 MB 26.5 MB/s eta 0:00:01
     ---------------------------------------- 7.4/7.4 MB 21.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13566 sha256=bc398eb4bc36f86a22f48b545828f0258bd956f6b72851320ff085af14a33cd1
  Stored in directory: c:\users\speec\appdata\local\pip\cache\wheels\a5\91\9f\00d66475960891a64867914273fcaf78df6cb04d905b104a2a
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-an

In [2]:
# 필요한 라이브러리 임포트
from GoogleNews import GoogleNews
from newspaper import Article
import pandas as pd
from transformers import pipeline
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
from datetime import datetime, timedelta


In [3]:
class NewsCrawler:
    def __init__(self, query, period='6m', lang='ko'):
        self.query = query
        self.period = period
        self.lang = lang
        
    def collect_google_news(self):
        """구글 뉴스에서 기사 수집"""
        googlenews = GoogleNews(lang=self.lang)
        googlenews.set_period(self.period)
        googlenews.search(self.query)
        return googlenews.result()
    
    def collect_naver_news(self, pages=5):
        """네이버 뉴스에서 기사 수집"""
        articles = []
        
        for page in range(1, pages + 1):
            url = f"https://search.naver.com/search.naver?where=news&query={self.query}&start={page*10-9}"
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            news_items = soup.select('.news_wrap')
            for item in news_items:
                try:
                    title = item.select_one('.news_tit').text
                    link = item.select_one('.news_tit')['href']
                    desc = item.select_one('.news_dsc').text if item.select_one('.news_dsc') else ""
                    
                    articles.append({
                        'title': title,
                        'description': desc,
                        'link': link
                    })
                except:
                    continue
                    
        return articles
    
    def parse_article(self, url):
        """기사 URL에서 본문 추출"""
        try:
            article = Article(url, language=self.lang)
            article.download()
            article.parse()
            return {
                "title": article.title,
                "text": article.text,
                "url": url
            }
        except:
            return None


In [4]:
class CommunityCrawler:
    def __init__(self):
        self.driver = None
        
    def setup_driver(self):
        """셀레니움 드라이버 설정"""
        if not self.driver:
            service = Service(ChromeDriverManager().install())
            self.driver = webdriver.Chrome(service=service)
            
    def close_driver(self):
        """드라이버 종료"""
        if self.driver:
            self.driver.quit()
            self.driver = None
            
    def collect_dcinside(self, gallery_id, pages=3):
        """디시인사이드 갤러리 게시글 수집"""
        self.setup_driver()
        posts = []
        
        for page in range(1, pages + 1):
            url = f"https://gall.dcinside.com/board/lists/?id={gallery_id}&page={page}"
            self.driver.get(url)
            time.sleep(2)
            
            # 게시글 목록 추출
            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            items = soup.select('.us-post')
            
            for item in items:
                try:
                    title = item.select_one('.gall_tit').text.strip()
                    link = item.select_one('.gall_tit a')['href']
                    date = item.select_one('.gall_date')['title']
                    
                    # 게시글 내용 가져오기
                    self.driver.get(f"https://gall.dcinside.com{link}")
                    time.sleep(1)
                    post_soup = BeautifulSoup(self.driver.page_source, 'html.parser')
                    content = post_soup.select_one('.writing_view_box').text.strip()
                    
                    posts.append({
                        'title': title,
                        'content': content,
                        'date': date,
                        'url': f"https://gall.dcinside.com{link}"
                    })
                except:
                    continue
                    
        return posts
    
    def collect_naver_cafe(self, cafe_id, menu_id, query, pages=3):
        """네이버 카페 게시글 수집"""
        self.setup_driver()
        posts = []
        
        for page in range(1, pages + 1):
            url = f"https://cafe.naver.com/{cafe_id}?iframe_url=/ArticleSearchList.nhn?search.clubid={cafe_id}&search.menuid={menu_id}&search.media=0&search.searchdate=all&search.defaultValue=1&search.exact=&search.include=&userDisplay=15&search.exclude=&search.option=0&search.sortBy=date&search.searchBy=0&search.searchBlockYn=0&search.includeAll=&search.query={query}&search.viewtype=title&search.page={page}"
            self.driver.get(url)
            time.sleep(2)
            
            # iframe 전환
            self.driver.switch_to.frame('cafe_main')
            
            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            items = soup.select('.article-board tr:not(.notice)')
            
            for item in items:
                try:
                    title = item.select_one('.article').text.strip()
                    link = item.select_one('.article')['href']
                    date = item.select_one('.date').text.strip()
                    
                    posts.append({
                        'title': title,
                        'date': date,
                        'url': f"https://cafe.naver.com{link}"
                    })
                except:
                    continue
                    
        return posts


In [5]:
def extract_entities(texts, model_name="jinmang2/klue-ner"):
    """텍스트에서 개체명(인물, 조직 등) 추출"""
    ner = pipeline("ner", model=model_name, tokenizer=model_name)
    
    entities = []
    for text in texts:
        try:
            ner_results = ner(text)
            for ent in ner_results:
                if ent['entity'] in ["PER", "ORG"]:  # 인물/조직만 필터링
                    entities.append({
                        "name": ent['word'],
                        "type": ent['entity'],
                        "context": text[:200]  # 앞 200자 문맥
                    })
        except:
            continue
            
    return entities

def save_to_csv(data, filename, encoding="utf-8-sig"):
    """데이터를 CSV 파일로 저장"""
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, encoding=encoding)
    print(f"파일 저장 완료: {filename}")
    return df


In [9]:
# 사용 예제 (디버깅 개선 버전)

# 1. 검색어 설정
query = "검색하고 싶은 키워드"  # 예: "SKT 유심 유출"
print(f"검색어: {query}")

# 2. 뉴스 수집 (디버깅 추가)
news_crawler = NewsCrawler(query)

# 네이버 뉴스 수집
print("네이버 뉴스 수집 시작...")
naver_news = news_crawler.collect_naver_news(pages=5)
print(f"네이버 뉴스 {len(naver_news)}개 수집 완료")

# 수집된 뉴스가 없는 경우 원인 확인
if len(naver_news) == 0:
    print("⚠️ 네이버 뉴스가 수집되지 않았습니다. 다음 사항을 확인하세요:")
    print("1. 검색어가 올바른지 확인")
    print("2. 네트워크 연결 상태 확인")
    print("3. 네이버 뉴스 사이트 접근 가능 여부 확인")
    
    # 테스트용 검색어로 재시도
    test_query = "삼성전자"
    print(f"\n테스트용 검색어 '{test_query}'로 재시도...")
    test_crawler = NewsCrawler(test_query)
    test_news = test_crawler.collect_naver_news(pages=1)
    print(f"테스트 결과: {len(test_news)}개 수집")

# 3. 유튜브 영상 정보 수집 (디버깅 추가)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def collect_youtube_videos(query, max_results=20):
    """유튜브에서 검색어로 영상 정보 수집 (디버깅 개선)"""
    print(f"유튜브 검색 시작: {query}")
    driver = webdriver.Chrome()
    youtube_data = []
    
    try:
        # 유튜브 검색
        search_url = f"https://www.youtube.com/results?search_query={quote(query)}"
        print(f"유튜브 URL: {search_url}")
        driver.get(search_url)
        time.sleep(3)
        
        # 페이지 로드 확인
        page_title = driver.title
        print(f"페이지 제목: {page_title}")
        
        # 스크롤하여 더 많은 영상 로드
        print("페이지 스크롤 중...")
        for i in range(3):
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(2)
            print(f"스크롤 {i+1}/3 완료")
        
        # 영상 정보 추출
        print("영상 요소 검색 중...")
        videos = driver.find_elements(By.CSS_SELECTOR, "div#contents ytd-video-renderer")
        print(f"발견된 영상 요소 수: {len(videos)}")
        
        if len(videos) == 0:
            print("⚠️ 영상 요소를 찾을 수 없습니다. CSS 선택자를 확인합니다...")
            # 대안 선택자 시도
            alternative_videos = driver.find_elements(By.CSS_SELECTOR, "ytd-video-renderer")
            print(f"대안 선택자로 발견된 영상: {len(alternative_videos)}")
            videos = alternative_videos
        
        for i, video in enumerate(videos[:max_results]):
            try:
                print(f"영상 {i+1} 처리 중...")
                title_element = video.find_element(By.CSS_SELECTOR, "a#video-title")
                title = title_element.get_attribute("title")
                video_url = title_element.get_attribute("href")
                
                if not title or not video_url:
                    print(f"영상 {i+1}: 제목 또는 URL이 없음")
                    continue
                
                # 채널명
                try:
                    channel = video.find_element(By.CSS_SELECTOR, "a.yt-simple-endpoint.style-scope.yt-formatted-string").text
                except:
                    channel = "채널명 없음"
                
                # 조회수와 업로드 시간
                try:
                    meta_info = video.find_elements(By.CSS_SELECTOR, "span.style-scope.ytd-video-meta-block")
                    views = meta_info[0].text if len(meta_info) > 0 else "정보 없음"
                    upload_time = meta_info[1].text if len(meta_info) > 1 else "정보 없음"
                except:
                    views = "정보 없음"
                    upload_time = "정보 없음"
                
                youtube_data.append({
                    'title': title,
                    'channel': channel,
                    'views': views,
                    'upload_time': upload_time,
                    'url': video_url,
                    'platform': 'YouTube'
                })
                
                print(f"영상 {i+1} 수집 완료: {title[:50]}...")
                
            except Exception as e:
                print(f"영상 {i+1} 처리 중 오류: {e}")
                continue
                
    except Exception as e:
        print(f"유튜브 크롤링 전체 오류: {e}")
    finally:
        driver.quit()
    
    return youtube_data

# 유튜브 영상 정보 수집
youtube_videos = collect_youtube_videos(query, max_results=30)
print(f"유튜브 영상 {len(youtube_videos)}개 수집 완료")

# 수집된 유튜브 영상이 없는 경우 원인 확인
if len(youtube_videos) == 0:
    print("⚠️ 유튜브 영상이 수집되지 않았습니다. 다음 사항을 확인하세요:")
    print("1. 검색어가 영어나 한글로 올바르게 입력되었는지 확인")
    print("2. Chrome 드라이버가 올바르게 설치되었는지 확인")
    print("3. 유튜브 사이트 접근 가능 여부 확인")

# 4. 수집된 데이터 통합 및 처리
all_texts = []
all_data = []

print(f"\n수집된 데이터 처리 시작...")
print(f"네이버 뉴스: {len(naver_news)}개")
print(f"유튜브 영상: {len(youtube_videos)}개")

# 네이버 뉴스 데이터 처리
successful_articles = 0
for i, article in enumerate(naver_news):
    print(f"뉴스 기사 {i+1}/{len(naver_news)} 처리 중...")
    if 'link' in article:
        parsed = news_crawler.parse_article(article['link'])
        if parsed and parsed['text']:
            all_texts.append(parsed['text'])
            all_data.append({
                'type': '네이버뉴스',
                'title': article.get('title', ''),
                'content': parsed['text'][:500],  # 앞 500자만 저장
                'date': article.get('date', ''),
                'url': article.get('link', ''),
                'source': article.get('source', '')
            })
            successful_articles += 1

print(f"성공적으로 처리된 뉴스 기사: {successful_articles}개")

# 유튜브 영상 데이터 처리 (제목과 채널명을 텍스트로 활용)
for i, video in enumerate(youtube_videos):
    print(f"유튜브 영상 {i+1}/{len(youtube_videos)} 처리 중...")
    video_text = f"{video['title']} {video['channel']}"
    all_texts.append(video_text)
    all_data.append({
        'type': '유튜브',
        'title': video['title'],
        'content': f"채널: {video['channel']}, 조회수: {video['views']}, 업로드: {video['upload_time']}",
        'date': video['upload_time'],
        'url': video['url'],
        'source': video['channel']
    })

print(f"전체 처리된 텍스트 수: {len(all_texts)}개")

# 5. 개체명 추출 (수집된 데이터가 있을 때만)
entities = []
if len(all_texts) > 0:
    print("개체명 추출 중...")
    try:
        # 더 안정적인 NER 모델 사용
        model_alternatives = [
            "klue/roberta-large-ner",  # KLUE 공식 모델
            "monologg/kobert-ner",     # KoBERT NER
            "jinmang2/korean-ner"      # 대안 모델
        ]
        
        for model_name in model_alternatives:
            try:
                print(f"NER 모델 시도: {model_name}")
                entities = extract_entities(all_texts, model_name)
                print(f"개체명 추출 성공: {len(entities)}개")
                break
            except Exception as e:
                print(f"모델 {model_name} 실패: {e}")
                continue
                
    except Exception as e:
        print(f"개체명 추출 실패: {e}")
        print("개체명 추출 없이 계속 진행합니다.")
else:
    print("⚠️ 처리할 텍스트가 없어 개체명 추출을 건너뜁니다.")

# 6. 결과 저장 (데이터가 있을 때만)
if len(all_data) > 0:
    print("결과 파일 저장 중...")
    # 전체 수집 데이터 저장
    save_to_csv(all_data, f"{query}_수집데이터.csv")
    
    # 네이버 뉴스만 따로 저장
    if len(naver_news) > 0:
        save_to_csv(naver_news, f"{query}_네이버뉴스.csv")
    
    # 유튜브 영상만 따로 저장
    if len(youtube_videos) > 0:
        save_to_csv(youtube_videos, f"{query}_유튜브영상.csv")
    
    # 추출된 개체명 저장
    if len(entities) > 0:
        save_to_csv(entities, f"{query}_개체명.csv")
else:
    print("⚠️ 저장할 데이터가 없습니다.")

# 7. 수집 결과 요약
print("\n" + "="*50)
print("=== 수집 결과 요약 ===")
print("="*50)
print(f"검색어: {query}")
print(f"네이버 뉴스: {len(naver_news)}개")
print(f"유튜브 영상: {len(youtube_videos)}개")
print(f"추출된 개체명: {len(entities)}개")
print(f"전체 텍스트 수: {len(all_texts)}개")
print(f"성공적으로 처리된 기사: {successful_articles}개")

# 수집 결과가 0인 경우 문제 진단
if len(naver_news) == 0 and len(youtube_videos) == 0:
    print("\n" + "!"*50)
    print("⚠️ 수집된 데이터가 없습니다!")
    print("!"*50)
    print("문제 해결 방법:")
    print("1. 검색어를 더 일반적인 키워드로 변경해보세요")
    print("2. 네트워크 연결을 확인하세요")
    print("3. 웹드라이버(Chrome)가 제대로 설치되었는지 확인하세요")
    print("4. 웹사이트 접근이 차단되지 않았는지 확인하세요")
    print("5. 시간을 두고 다시 시도해보세요")

# 개체명 유형별 통계
if len(entities) > 0:
    entity_types = {}
    for entity in entities:
        entity_type = entity.get('type', 'Unknown')
        entity_types[entity_type] = entity_types.get(entity_type, 0) + 1

    print("\n=== 개체명 유형별 통계 ===")
    for etype, count in entity_types.items():
        print(f"{etype}: {count}개")
else:
    print("\n=== 개체명 통계 ===")
    print("추출된 개체명이 없습니다.")


검색어: 검색하고 싶은 키워드
네이버 뉴스 수집 시작...
네이버 뉴스 0개 수집 완료
⚠️ 네이버 뉴스가 수집되지 않았습니다. 다음 사항을 확인하세요:
1. 검색어가 올바른지 확인
2. 네트워크 연결 상태 확인
3. 네이버 뉴스 사이트 접근 가능 여부 확인

테스트용 검색어 '삼성전자'로 재시도...
테스트 결과: 0개 수집
유튜브 검색 시작: 검색하고 싶은 키워드
유튜브 크롤링 전체 오류: name 'quote' is not defined
유튜브 영상 0개 수집 완료
⚠️ 유튜브 영상이 수집되지 않았습니다. 다음 사항을 확인하세요:
1. 검색어가 영어나 한글로 올바르게 입력되었는지 확인
2. Chrome 드라이버가 올바르게 설치되었는지 확인
3. 유튜브 사이트 접근 가능 여부 확인

수집된 데이터 처리 시작...
네이버 뉴스: 0개
유튜브 영상: 0개
성공적으로 처리된 뉴스 기사: 0개
전체 처리된 텍스트 수: 0개
⚠️ 처리할 텍스트가 없어 개체명 추출을 건너뜁니다.
⚠️ 저장할 데이터가 없습니다.

=== 수집 결과 요약 ===
검색어: 검색하고 싶은 키워드
네이버 뉴스: 0개
유튜브 영상: 0개
추출된 개체명: 0개
전체 텍스트 수: 0개
성공적으로 처리된 기사: 0개

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
⚠️ 수집된 데이터가 없습니다!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
문제 해결 방법:
1. 검색어를 더 일반적인 키워드로 변경해보세요
2. 네트워크 연결을 확인하세요
3. 웹드라이버(Chrome)가 제대로 설치되었는지 확인하세요
4. 웹사이트 접근이 차단되지 않았는지 확인하세요
5. 시간을 두고 다시 시도해보세요

=== 개체명 통계 ===
추출된 개체명이 없습니다.


## 사용 시 주의사항

1. **웹사이트 정책 준수**
   - 각 웹사이트의 이용약관과 로봇 정책을 확인하고 준수해야 합니다.
   - 과도한 요청은 IP 차단의 원인이 될 수 있습니다.

2. **시간 간격 설정**
   - 웹사이트 서버에 부담을 주지 않도록 적절한 시간 간격을 두고 크롤링합니다.
   - 현재 코드에는 기본적인 대기 시간이 설정되어 있습니다.

3. **로그인이 필요한 경우**
   - 네이버 카페 등 로그인이 필요한 사이트는 별도의 로그인 처리가 필요할 수 있습니다.
   - Selenium을 사용하여 수동으로 로그인 후 크롤링을 진행하세요.

4. **데이터 저장 및 관리**
   - 수집된 데이터는 개인정보 보호법을 준수하여 관리해야 합니다.
   - 민감한 정보가 포함된 경우 적절한 보안 조치를 취하세요.
